## Agent Executor

In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.35.0"

add local maven repository

In [2]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos


Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Repositories count: 5
name: central url: https://repo.maven.apache.org/maven2/ release:true update:never snapshot:false update:never 
name: jcenter url: https://jcenter.bintray.com/ release:true update:never snapshot:false update:never 
name: jboss url: https://repository.jboss.org/nexus/content/repositories/releases/ release:true update:never snapshot:false update:never 
name: atlassian url: https://packages.atlassian.com/maven/public release:true update:never snapshot:false update:never 
name: local url: file:///Users/bsorrentino/.m2/repository/ release:true update:never snapshot:true update:always 


Remove installed package from Jupiter cache

In [3]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

Install required maven dependencies

In [4]:
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-SNAPSHOT
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:1.0-SNAPSHOT
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:1.0-SNAPSHOT
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}

%dependency /resolve

Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-langchain4j:1.0-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-agent-executor:1.0-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:0.35.0
Adding dependency dev.langchain4j:langchain4j-open-ai:0.35.0
Solving dependencies
Resolved artifacts count: 27
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core-jdk8/1.0-SNAPSHOT/langgraph4j-core-jdk8-1.0-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cach

Initialize Logger

In [5]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    var lm = java.util.logging.LogManager.getLogManager();
    lm.checkAccess(); 
    lm.readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AgentExecutor");


In [6]:
import dev.langchain4j.agent.tool.P;
import dev.langchain4j.agent.tool.Tool;

import java.util.Optional;

import static java.lang.String.format;

public class TestTool {
    private String lastResult;

    Optional<String> lastResult() {
        return Optional.ofNullable(lastResult);
    }

    @Tool("tool for test AI agent executor")
    String execTest(@P("test message") String message) {

        lastResult = format( "test tool executed: %s", message);
        return lastResult;
    }
}


In [7]:
import static org.bsc.langgraph4j.StateGraph.END;
import static org.bsc.langgraph4j.StateGraph.START;
import org.bsc.langgraph4j.CompileConfig;
import org.bsc.langgraph4j.RunnableConfig;
import org.bsc.langgraph4j.checkpoint.BaseCheckpointSaver;
import org.bsc.langgraph4j.checkpoint.MemorySaver;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.serializer.StateSerializer;

import org.bsc.langgraph4j.agentexecutor.AgentExecutor;

import dev.langchain4j.model.openai.OpenAiChatModel;

import java.util.List;
import java.util.Map;
import java.util.stream.Collectors;



var chatLanguageModel = OpenAiChatModel.builder()
    .apiKey( System.getenv("OPENAI_API_KEY") )
    //.modelName( "gpt-3.5-turbo-0125" )
    .modelName( "gpt-4o-mini" )
    .logResponses(true)
    .maxRetries(2)
    .temperature(0.0)
    .maxTokens(2000)
    .build();

var agentExecutor = new AgentExecutor();

var stateGraph = agentExecutor.graphBuilder()
        .chatLanguageModel(chatLanguageModel)
        .objectsWithTools(List.of(new TestTool()))
        .build();




### Test 1 
Update State replacing the 'input'

In [8]:
var saver = new MemorySaver();

CompileConfig compileConfig = CompileConfig.builder()
                .checkpointSaver( saver )
                .build();

var graph = stateGraph.compile( compileConfig );

var config = RunnableConfig.builder()
                .threadId("test1")
                .build();    
                
var iterator = graph.streamSnapshots( Map.of( "input", "perform test once" ), config );  

for( var step : iterator ) {
    log.info( "STEP: {}", step );
}



2024-10-11 13:05:40 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> START
2024-10-11 13:05:40 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor callAgent callAgent
2024-10-11 13:05:42 INFO REPL.$JShell$37 do_it$ STEP: NodeOutput{node=__START__, state={input=perform test once, intermediate_steps=[]}}
2024-10-11 13:05:42 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor executeTools executeTools
2024-10-11 13:05:42 FINEST org.bsc.langgraph4j.langchain4j.tool.ToolNode execute execute: execTest
2024-10-11 13:05:42 INFO REPL.$JShell$37 do_it$ STEP: StateSnapshot{node=agent, state={input=perform test once, intermediate_steps=[], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_IWpeVQWcA0B2GJzXbQiLCQHL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}, config=RunnableConfig(threadId=test1, checkPointId=dbe479ed-21dd-4e9b-bef8-3f4fda0abdb9, nextNode=action, streamMode=SNAPS

In [9]:
var history = graph.getStateHistory(config).stream().collect( Collectors.toList() );

var state2 =  history.get(2);

var updatedConfig = graph.updateState( state2.config(), Map.of( "input", "perform test twice"), null);

var iterator = graph.streamSnapshots( null, updatedConfig );  

for( var step : iterator ) {
    log.info( "STEP:\n {}", step );
}


2024-10-11 13:05:43 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME REQUEST
2024-10-11 13:05:43 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME FROM agent
2024-10-11 13:05:43 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor executeTools executeTools
2024-10-11 13:05:43 FINEST org.bsc.langgraph4j.langchain4j.tool.ToolNode execute execute: execTest
2024-10-11 13:05:43 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor callAgent callAgent
2024-10-11 13:05:44 INFO REPL.$JShell$41 do_it$ STEP:
 StateSnapshot{node=action, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_IWpeVQWcA0B2GJzXbQiLCQHL", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], observation=test tool executed: perform test once]], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_IWpeVQWcA0B2GJzX

### Test 2
Update State replacing the 'input' using a cloned state

In [10]:

var config = RunnableConfig.builder()
                .threadId("test2")
                .build();    
                
var iterator = graph.streamSnapshots( Map.of( "input", "perform test once" ), config );  

for( var step : iterator ) {
    log.info( "STEP: {}", step );
}



2024-10-11 13:05:46 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> START
2024-10-11 13:05:46 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor callAgent callAgent
2024-10-11 13:05:47 INFO REPL.$JShell$42 do_it$ STEP: NodeOutput{node=__START__, state={input=perform test once, intermediate_steps=[]}}
2024-10-11 13:05:47 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor executeTools executeTools
2024-10-11 13:05:47 FINEST org.bsc.langgraph4j.langchain4j.tool.ToolNode execute execute: execTest
2024-10-11 13:05:47 INFO REPL.$JShell$42 do_it$ STEP: StateSnapshot{node=agent, state={input=perform test once, intermediate_steps=[], agent_outcome=AgentOutcome[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_BKVXOUlQmFPqydEJZ3hotT0H", name = "execTest", arguments = "{"message":"perform test once"}" }, log=], finish=null]}, config=RunnableConfig(threadId=test2, checkPointId=c470d430-fed3-477c-a6a7-84c6e67c55e4, nextNode=action, streamMode=SNAPS

In [13]:
var history = graph.getStateHistory(config).stream().collect( Collectors.toList() );

var state2 =  history.get(2);

var updatedState = new HashMap<String,Object>();
updatedState.putAll(state2.state().data());

//System.out.println( state2.state().data() );
updatedState.put(  "input", "perform test twice"  );
//System.out.println( updatedState );

var updatedConfig = graph.updateState( state2.config(), updatedState );
log.info( "UPDATED CONFIG: {}", updatedConfig );

var iterator = graph.streamSnapshots( null, updatedConfig );  

try {
    for( var step : iterator ) {
        log.info( "STEP:\n {}", step );
    }    
}
catch( Exception e ) {
    Throwable t = e ;
    while( t.getCause() != null ) {
        t = t.getCause();
    }
    t.printStackTrace();
}


2024-10-11 13:06:10 INFO REPL.$JShell$49 do_it$ UPDATED CONFIG: RunnableConfig(threadId=test2, checkPointId=c470d430-fed3-477c-a6a7-84c6e67c55e4, nextNode=null, streamMode=VALUES)
2024-10-11 13:06:10 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME REQUEST
2024-10-11 13:06:10 FINEST org.bsc.langgraph4j.CompiledGraph$AsyncNodeGenerator <init> RESUME FROM agent
2024-10-11 13:06:10 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor executeTools executeTools
2024-10-11 13:06:10 FINEST org.bsc.langgraph4j.langchain4j.tool.ToolNode execute execute: execTest
2024-10-11 13:06:10 FINEST org.bsc.langgraph4j.agentexecutor.AgentExecutor callAgent callAgent
2024-10-11 13:06:11 INFO REPL.$JShell$50 do_it$ STEP:
 StateSnapshot{node=action, state={input=perform test twice, intermediate_steps=[IntermediateStep[action=AgentAction[toolExecutionRequest=ToolExecutionRequest { id = "call_BKVXOUlQmFPqydEJZ3hotT0H", name = "execTest", arguments = "{"message":"perform test once"}" 